In [ ]:
# https://www.analyticsvidhya.com/blog/2020/04/5-amazing-google-colab-hacks-you-should-try-today/
# You can also try to use the reduce_mem_usage method from this link => https://www.kaggle.com/artgor/elo-eda-and-models
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Let's train a custom ensemble regressor. Please refer to the below instructions for the same.

Custom Stacking regressor:

a) Split your whole data into train and test(80-20)

b) Now, in the 80% train set, split the train set into D1 and D2.(50-50).

From this D1, do sampling with replacement to create d1,d2,d3....dk(k samples).

Now create 'k' models and train each of these models with each of these k samples.

c) Now pass the D2 set to each of these k models; now, you will get k predictions for D2 from each of these models.

d) Now, using these k predictions, create a new dataset, for D2, you already know it's corresponding target values, so now you train a metamodel with these k predictions.

e) Now, for model evaluation, you can use the 20% data that you have kept as the test set. Pass that test set to each of the base models, and you will get 'k' predictions. Now you create a new dataset with these k predictions and pass it to your metamodel, and you will get the final prediction. Using this final prediction and the targets for the test set, you can calculate the model's performance score.

 

I hope you got it. For base models, you can consider any combination of models; you can take the DecisionTreeRegressor, Linear Regression, SVR, SVR with different kernels, RF, XGBoost, etc. And here, you can tune for the number of base models. The procedure mentioned above requires you to do custom coding. There are no library functions available for this as it is a custom model implementation.



In [ ]:
https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/

In [ ]:
import numpy as np
import pandas as pd
#import dask.dataframe as dd
# https://www.kaggle.com/greentearus/bimbo-w-dask-and-callbacks
final_types = {'WeekNumber': np.uint8,
                'SalesDepotID': np.uint16,
                'SalesChannelID':np.uint8,
                'RouteID':np.uint16,
                'ClientID':np.uint32,
                'ProductID':np.uint16,
                'Demand':np.uint8,
                'NewClientName': np.object,
                'NewProductName': np.object,
                'pieces':np.float64,
                'weight':np.float64,
                'brand':np.object,
                'Town':np.object,
                'State':np.object
                }

In [ ]:
data=pd.read_csv('/content/drive/My Drive/Colab Notebooks/final_df.csv',sep = ",", usecols = final_types.keys(), dtype = final_types)

In [ ]:
data.shape

(74174653, 14)

In [ ]:
data.columns

Index(['WeekNumber', 'SalesDepotID', 'SalesChannelID', 'RouteID', 'ClientID',
       'ProductID', 'Demand', 'NewClientName', 'NewProductName', 'pieces',
       'weight', 'brand', 'Town', 'State'],
      dtype='object')

In [ ]:
'''
https://www.datacamp.com/community/tutorials/python-select-columns
loc: label-based
iloc: integer position-based
'''

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74174653 entries, 0 to 74174652
Data columns (total 14 columns):
 #   Column          Dtype  
---  ------          -----  
 0   WeekNumber      uint8  
 1   SalesDepotID    uint16 
 2   SalesChannelID  uint8  
 3   RouteID         uint16 
 4   ClientID        uint32 
 5   ProductID       uint16 
 6   Demand          uint8  
 7   NewClientName   object 
 8   NewProductName  object 
 9   pieces          float64
 10  weight          float64
 11  brand           object 
 12  Town            object 
 13  State           object 
dtypes: float64(2), object(5), uint16(3), uint32(1), uint8(3)
memory usage: 4.8+ GB


In [ ]:
print ('Dropping duplicates! - same customer may buy same product from the same SalesDepotID - If the same data repeats, we can drop the dupliactes here')
#final_data= final_data.loc[:,].drop_duplicates(subset=['WeekNumber','SalesDepotID', 'ClientID', 'ProductID'])
'''
TRAIN_Semana_3 = FULL_train.loc[FULL_train['Semana'] == 3, ['Agencia_ID', 'Producto_ID', 'Cliente_ID', 'Demanda_uni_equil']].
drop_duplicates(subset=['Agencia_ID', 'Producto_ID', 'Cliente_ID'])
'''
final_data= data.loc[data['WeekNumber']==8,['WeekNumber','SalesDepotID','SalesChannelID','RouteID','ClientID','ProductID','Demand','NewClientName','NewProductName','pieces','weight','brand','Town','State']].drop_duplicates(subset=['WeekNumber','SalesDepotID', 'ClientID', 'ProductID'])

Dropping duplicates! - same customer may buy same product from the same SalesDepotID - If the same data repeats, we can drop the dupliactes here


In [ ]:
final_data.shape

(10387527, 14)

In [ ]:
del data

In [ ]:
#1000000 -- it took 1m 7secs
final_data1=final_data[:9000000]

In [ ]:
final_data1.shape

(9000000, 14)

In [ ]:
del final_data

In [ ]:
final_data1['Demand'] = final_data1['Demand'].apply(lambda x:np.log(x + 1))

In [ ]:
final_data1.head(2)

,WeekNumber,SalesDepotID,SalesChannelID,RouteID,ClientID,ProductID,Demand,NewClientName,NewProductName,pieces,weight,brand,Town,State
53360774,8,1110,7,3301,15766,1212,1.609438,Post,rol canel,-0.775786,-0.437010,BIM,2008 AG. LAGO FILT,"MÉXICO, D.F."
53360775,8,1110,7,3301,15766,1216,1.791759,Post,rol glass,-0.775786,-0.373384,BIM,2008 AG. LAGO FILT,"MÉXICO, D.F."


In [ ]:
final_data1.columns

Index(['WeekNumber', 'SalesDepotID', 'SalesChannelID', 'RouteID', 'ClientID',
       'ProductID', 'Demand', 'NewClientName', 'NewProductName', 'pieces',
       'weight', 'brand', 'Town', 'State'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import StandardScaler
'''
scaler_pieces = StandardScaler()
scaler_pieces.fit(final_data1['pieces'].values.reshape(-1,1))
final_data1['pieces'] = scaler_pieces.transform(final_data1['pieces'].values.reshape(-1,1))
scaler_weight = StandardScaler()
scaler_weight.fit(final_data1['weight'].values.reshape(-1,1))
final_data1['weight'] = scaler_weight.transform(final_data1['weight'].values.reshape(-1,1))
'''

In [ ]:
from sklearn.preprocessing import LabelEncoder
final_data1["NewClientName"] = LabelEncoder().fit_transform( final_data1['NewClientName'] )
final_data1["NewProductName"] = LabelEncoder().fit_transform( final_data1['NewProductName'] )
final_data1["Town"] = LabelEncoder().fit_transform( final_data1['Town'] )
final_data1["State"] = LabelEncoder().fit_transform( final_data1['State'] )
final_data1["brand"] = LabelEncoder().fit_transform( final_data1['brand'] )

In [ ]:
final_data1.shape

(9000000, 14)

In [ ]:
final_data1.columns

Index(['WeekNumber', 'SalesDepotID', 'SalesChannelID', 'RouteID', 'ClientID',
       'ProductID', 'Demand', 'NewClientName', 'NewProductName', 'pieces',
       'weight', 'brand', 'Town', 'State'],
      dtype='object')

In [ ]:
final_data1.head(5)

,WeekNumber,SalesDepotID,SalesChannelID,RouteID,ClientID,ProductID,Demand,NewClientName,NewProductName,pieces,weight,brand,Town,State
53360774,8,1110,7,3301,15766,1212,1.609438,11,412,-0.775786,-0.437010,1,5,16
53360775,8,1110,7,3301,15766,1216,1.791759,11,413,-0.775786,-0.373384,1,5,16
53360776,8,1110,7,3301,15766,1220,0.693147,11,153,0.095969,-0.394593,1,5,16
53360777,8,1110,7,3301,15766,1238,1.386294,11,359,-0.775786,-0.352175,1,5,16
53360778,8,1110,7,3301,15766,1240,1.098612,11,274,-0.339909,-0.415801,1,5,16


In [ ]:
from sklearn.model_selection import train_test_split
Y=final_data1['Demand']
X=final_data1.drop(['Demand'],axis=1)

In [ ]:
#split whole data into train & test - 80:20
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size=0.2,random_state=24)
print("="*100)
print("x_train:",x_train.shape)
print("y_train:",y_train.shape)
print("x_test:",x_test.shape)
print("y_test:",y_test.shape)
print("="*100)

x_train: (800000, 13)
y_train: (800000,)
x_test: (200000, 13)
y_test: (200000,)


In [ ]:
'''
https://pynative.com/python-random-sample/
https://www.geeksforgeeks.org/how-to-randomly-select-rows-from-pandas-dataframe/
random.choices(seq, n)	Generate n samples from a sequence with the possibility of repetition. Sample with replacement
random.sample(seq, n)	Generate n unique samples (multiple items) from a sequence without repetition. Here, A seq can be a list, set, string, tuple. Sample without replacement.
'''

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

In [ ]:
https://www.kdnuggets.com/2019/01/ensemble-learning-5-main-approaches.html
https://www.section.io/engineering-education/ensemble-learning-based-regression-model-using-python/.
https://machinelearningmastery.com/stacking-ensemble-machine-learning-with-python/
https://github.com/Anwoy-p/case_study_1/blob/master/temp_CS_1.ipynb

In [ ]:
def base_model1(X_data,Y_data):
    params = {
     'max_depth':[6,8,10,12,15,20],
     'min_samples_split': [5, 10, 100]}

    DT=DecisionTreeRegressor()

    clf = GridSearchCV(DT, params, cv=2, scoring='neg_root_mean_squared_error')

    clf.fit(X_data,Y_data)
    
    bst=clf.best_params_

    #print("feature importance:",clf.best_estimator_.feature_importances_)
    #feature importance: [0.         0.00728032 0.00371949 0.27052722 0.02523803 0.07258978
    #0.00609322 0.120515   0.0432022  0.41076663 0.01531697 0.00676692 0.01798423]
    
    model = DecisionTreeRegressor(
    max_depth=bst['max_depth'],
    min_samples_split=bst['min_samples_split'], 
    random_state=42)

    model.fit(
    X_data,Y_data)
    
    return model

In [ ]:
import numpy as np
def custom_ensemble(x_train,y_train,x_test,y_test,n_estimators):
  d1_train,d2_test,d1_ytrain,d2_ytest =train_test_split(x_train,y_train,test_size=0.50)
  #print("d1_train:",d1_train.shape)
  #print("d1_ytrain:",d1_ytrain.shape)
  #print("d2_test:",d2_test.shape)
  #print("d2_ytest:",d2_ytest.shape)
  #print(type(d1_train))
  list_of_all_models=[]
  d2_prediction=[]
  test_prediction=[]

  for i in range(n_estimators):
    #print("name:",name)
    selecting_rows=np.random.choice(len(d1_train),int(0.4*len(d1_train)),replace=False)
    sample_x=d1_train.iloc[selecting_rows]
    sample_y=d1_ytrain.iloc[selecting_rows]
    #print("sample_x:",sample_x.shape)
    #print("sample_y:",sample_y.shape)
    models1=base_model1(sample_x,sample_y)
    list_of_all_models.append(models1)
    pred=models1.predict(d2_test)
    d2_prediction.append(pred.reshape(-1,1)) 
    test= models1.predict(x_test)
    test_prediction.append(test.reshape(-1,1))
  import pickle
  pickle.dump((list_of_all_models),open('/content/drive/My Drive/Colab Notebooks/list_of_all_models.pkl','wb'))
  #list_of_all_models= pickle.load(open('/content/drive/My Drive/Colab Notebooks/list_of_all_models.pkl', 'rb'))
  D_meta = np.hstack(d2_prediction)
  #print("D_meta.shape:",D_meta.shape)
  D_meta1=pd.DataFrame(D_meta)
  print("D_meta1.shape:",D_meta1.shape)
  from sklearn.linear_model import LinearRegression
  meta_model = LinearRegression()
  meta_model.fit(D_meta1, d2_ytest)
  D_meta_test = np.hstack(test_prediction)
  D_meta1_test=pd.DataFrame(D_meta_test)
  print("D_meta1_test:",D_meta1_test.shape)
  ytest_pred=meta_model.predict(D_meta1_test)
  #ytest_pred=meta_model.predict(D_meta1)
  from sklearn.metrics import mean_squared_error as mse
  print("Root Mean Squared Log Error(RMSLE)",np.sqrt(mse(y_test,ytest_pred)))

In [ ]:
custom_ensemble(x_train,y_train,x_test,y_test,5)

feature importance: [0.         0.00728032 0.00371949 0.27052722 0.02523803 0.07258978
 0.00609322 0.120515   0.0432022  0.41076663 0.01531697 0.00676692
 0.01798423]
feature importance: [0.         0.00485606 0.00225673 0.27944747 0.02554658 0.10126393
 0.00525987 0.11988545 0.03139445 0.38666374 0.01422344 0.00594025
 0.02326203]
feature importance: [0.         0.00497872 0.00439959 0.27648897 0.02834599 0.07563702
 0.00414336 0.12103856 0.06402344 0.39284108 0.00238299 0.00563566
 0.02008463]
feature importance: [0.         0.00608601 0.00646866 0.27514711 0.0283526  0.06817767
 0.0059492  0.13440739 0.05360678 0.38953669 0.00550086 0.00615958
 0.02060746]
feature importance: [0.         0.00389108 0.0074341  0.27235649 0.02581691 0.09590072
 0.00602768 0.12998412 0.03401481 0.38926091 0.00814003 0.00624154
 0.0209316 ]
D_meta1.shape: (400000, 5)
D_meta1_test: (200000, 5)
Root Mean Squared Log Error(RMSLE) 0.5857926360508493
